In [210]:
import sys, os
# sys.path.insert(0, os.path.abspath("../src"))


In [211]:
ls

Dockerfile        config/           requirements.txt  template.py
LICENSE           dvc.yaml          research/         templates/
README.md         logs/             schema.yaml       venv/
app.py            main.py           setup.py          youtube/
artifacts/        params.yaml       src/


In [212]:
cd Users

[Errno 2] No such file or directory: 'Users'
/Users/gojuruakshith/YouTube-Fake-Thumbnail-Detector


In [213]:
ls

Dockerfile        config/           requirements.txt  template.py
LICENSE           dvc.yaml          research/         templates/
README.md         logs/             schema.yaml       venv/
app.py            main.py           setup.py          youtube/
artifacts/        params.yaml       src/


In [214]:
cd gojuruakshith/

[Errno 2] No such file or directory: 'gojuruakshith/'
/Users/gojuruakshith/YouTube-Fake-Thumbnail-Detector


In [215]:
ls

Dockerfile        config/           requirements.txt  template.py
LICENSE           dvc.yaml          research/         templates/
README.md         logs/             schema.yaml       venv/
app.py            main.py           setup.py          youtube/
artifacts/        params.yaml       src/


In [216]:
cd YouTube-Fake-Thumbnail-Detector/

[Errno 2] No such file or directory: 'YouTube-Fake-Thumbnail-Detector/'
/Users/gojuruakshith/YouTube-Fake-Thumbnail-Detector


In [217]:
%pwd

'/Users/gojuruakshith/YouTube-Fake-Thumbnail-Detector'

In [218]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [219]:
from mlProject.utils.common import read_yaml, create_directories
from mlProject.constants import CONFIG_FILE_PATH, PARAM_FILE_PATH, SCHEMA_FILE_PATH
from pathlib import Path

class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAM_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        # ensure artifacts_root is a Path
        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        root_dir = Path(config.root_dir)
        create_directories([root_dir])

        return DataIngestionConfig(
            root_dir=root_dir,
            source_url=str(config.source_url),
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )


In [220]:
import urllib.request as request
import zipfile
from pathlib import Path
from mlProject import logger
from mlProject.utils.common import get_size
import ssl
import certifi
import urllib.request as request

In [221]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
         if not self.config.local_data_file.exists():
              self.config.local_data_file.parent.mkdir(parents=True, exist_ok=True)
              ctx = ssl.create_default_context(cafile=certifi.where())

              with request.urlopen(self.config.source_url, context=ctx) as r, open(self.config.local_data_file, "wb") as f:
                 f.write(r.read())

              logger.info(f"Downloaded file to: {self.config.local_data_file}")
         else:
             logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        self.config.unzip_dir.mkdir(parents=True, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)


In [222]:
try:
    cm = ConfigurationManager()
    data_ingestion_config = cm.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2026-01-04 21:36:34,892: INFO: common: YAML file loaded successfully: config/config.yaml]
[2026-01-04 21:36:34,893: INFO: common: YAML file loaded successfully: params.yaml]
[2026-01-04 21:36:34,893: INFO: common: YAML file loaded successfully: schema.yaml]
[2026-01-04 21:36:34,894: INFO: common: Created directory at: artifacts]
[2026-01-04 21:36:34,894: INFO: common: Created directory at: artifacts/data_ingestion]
[2026-01-04 21:36:35,677: INFO: 3235098341: Downloaded file to: artifacts/data_ingestion/data.zip]
